In [16]:
import cobra as cb
import pandas as pd
import numpy as np
from cobra.util.solver import linear_reaction_coefficients

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 0)
pd.set_option('display.max_colwidth', None)

In [2]:
GROWTH_MIN_OBJ = 0.01

# Functions for breaking down iiFBA tasks

### Functions to clean and streamline
- write code for pFBA
- write seperate code for Sampling
- Write code for iteration
- Write code to re-initialize environment
	
	Lastly:
- Write wrapper to run all combined
	

# Lin. Alg. Based Functions


## Environment


In [3]:
def init_iifba(models, media, iterations, m_vals=[1,1]):
	# get list of all unique rxns and exchanges
	org_exs = set()
	org_rxns = set()
	for model in models:
		exs_set = set(model.exchanges.list_attr("id"))
		org_exs = org_exs | exs_set # exchanges
		rxns_set = set(model.reactions.list_attr("id"))
		org_rxns = org_rxns | rxns_set # reactions

	# initialize env
	rows = (iterations) * m_vals[0] * m_vals[1] + 1 # add one iteration for final env
	cols = len(org_exs)
	env_f = np.zeros((rows, cols))
	env0_masks = [np.array(list(org_exs)) == rxn_id for rxn_id in list(media.keys()) ]
	for flux_idx, flux in enumerate(list(media.values())):
		env_f[0][env0_masks[flux_idx]] = flux
	
	#set columns for multi-indexing
	iters_col = np.repeat(np.arange(1, iterations+1), m_vals[0] * m_vals[1]) 
	run_col = np.tile(np.arange(m_vals[0] * m_vals[1]), iterations)
	iters_col = np.insert(iters_col, 0, 0) # add 0th iteration
	run_col = np.insert(run_col, 0, 0) # add 0th run 
	multi_idx = [iters_col , run_col]
	env_f = pd.DataFrame(env_f, columns=list(org_exs), index=multi_idx) # convert to interprettable df
	env_f.index.names = ["Iteration", "Run"]

	# initialize org_fluxes
	rows = iterations * m_vals[0] * m_vals[1] * len(models)
	cols = len(org_rxns)
	org_F = np.zeros((rows, cols)) # pfba will drop run column
	
	# create unique multi-index for 
	models_col = np.tile(np.arange(len(models)), iterations * m_vals[0] * m_vals[1]) 
	iters_col = np.repeat(np.arange(iterations), m_vals[0] * m_vals[1] * len(models)) 
	run_col = np.tile(np.repeat(np.arange(m_vals[0] * m_vals[1]), len(models)), iterations) 
	multi_idx = [models_col, iters_col , run_col]
	org_F = pd.DataFrame(org_F, columns=list(org_rxns), index=multi_idx)	# convert to interprettable df
	org_F.index.names = ["model", "Iteration", "Run"]
	
	return env_f, org_F

In [4]:
def set_env(model, env_f, iter, run):
	for ex in model.exchanges:
		ex.lower_bound = env_f.loc[iter, run][ex.id]
	
	return model

## Optimization

In [5]:
def run_pfba(model, model_idx, iter, org_F):
	# run pFBA
	sol1 = model.slim_optimize()
	if sol1 > GROWTH_MIN_OBJ:
		sol = cb.flux_analysis.parsimonious.pfba(model)
		
		org_F.loc[(model_idx, iter, 0), list(sol.fluxes.index)] = sol.fluxes.values
	# do nothing otherwise - already initiated as zeros!

	return org_F

In [6]:
def run_sampling(model, model_idx, iter, org_F, m_vals, rep_idx, obj_percent):
	# ensure sample space is constrained above a certain objective value
	min_obj = model.slim_optimize() * obj_percent
	
	# set obj to be above min_obj
	obj_rxn = [rxn.id for rxn in linear_reaction_coefficients(model).keys()][0]
	model.reactions.get_by_id(obj_rxn).lower_bound = min_obj

	# run flux sampling
	if iter == 0:
		sample_ct = m_vals[0] * m_vals[1]
	else:
		sample_ct = m_vals[1]
	sol = cb.sampling.sample(model, sample_ct)
	
	# standardize and save output
	arrays = [[model_idx] * sample_ct, [iter] * sample_ct, list(sol.index + rep_idx * sample_ct)]
	tuples = list(zip(*arrays))
	multi_idx = pd.MultiIndex.from_tuples(tuples, names=['model', 'Iteration', 'Run'])
	sol.index = multi_idx
	
	org_F.loc[sol.index, sol.columns] = sol

	return org_F

	


## Flux Update

In [7]:
def update_pfba_env(env_f, org_F, flow, rel_abund, iter):
	# get initial env. for flow
	init_env = env_f.loc[0,0].to_numpy()
	#pull iter info
	env_tmp = env_f.loc[iter, 0][:].to_numpy()
	run_exs = org_F.loc[:, iter, 0][env_f.columns].to_numpy()
		
	# run update
	flux_sums = (run_exs.T @ rel_abund).flatten()
	env_f.loc[iter+1, 0] = (1-flow)*(env_tmp - flux_sums) + flow*init_env
	
	return env_f


In [8]:
def update_sampling_env(env_f, org_F, flow, rel_abund, iter, m_vals, Mi, rep_idx):
	# get initial env. for flow
	init_env = env_f.loc[(0,0)].to_numpy()

	sample_ct = m_vals[0] * m_vals[1] if iter == 0 else m_vals[1]
	for sample_idx in range(sample_ct):
		#pull run info
		env_tmp = env_f.loc[iter, Mi][:].to_numpy()
		run_exs = org_F.loc[:, iter, Mi][env_f.columns].to_numpy()

		# run update
		flux_sums = (run_exs.T @ rel_abund).flatten()
		env_f.loc[iter+1, sample_idx+ m_vals[1]*rep_idx] = (1-flow)*(env_tmp - flux_sums) + flow*init_env

	return env_f

## Wrapper Function


In [9]:
def iipfba(models, media, rel_abund=None,
		   iters=10, flow=0.5):
	env_fluxes, org_fluxes = init_iifba(models, media, iters)

	for iter in range(iters):
		print("Iteration:", iter)

		for org_idx, org_model in enumerate(models):
			with org_model as model:
				# set exchanges
				model = set_env(model, env_fluxes, iter, 0) # only 0 runs

				# run optim
				org_fluxes = run_pfba(model, org_idx, iter, org_fluxes)
				
		# update fluxes
		env_fluxes = update_pfba_env(env_fluxes, org_fluxes, flow, rel_abund, iter)

	# pfba has no use for Run index
	env_fluxes = env_fluxes.droplevel("Run")
	org_fluxes =org_fluxes.droplevel("Run")

	return env_fluxes, org_fluxes

In [10]:
def iisampling(models, media, rel_abund, iters=10, flow=0.5, m_vals=[1,1], objective_percent= 0.9):
	# mapping of what flux sampling to iterate
	M = np.zeros([m_vals[0],iters],dtype=int) #randomly pre-assign sampling initial point matrix
	for i in range(1, iters):
		Mcol = np.sort(np.random.choice(m_vals[0]*m_vals[1],m_vals[0],replace=False))
		M[:,i]=Mcol

	# initialize env and org fluxes
	env_fluxes, org_fluxes = init_iifba(models, media, iters, m_vals)

	for iter in range(iters):
		print("Iteration:", iter)

		# number of times to re-sample per iteration
		repeat_ct = 1 if iter == 0 else m_vals[0] 
		for rep_idx in range(repeat_ct):
			Mi = M[rep_idx, iter]

			# samples taken
			samples = m_vals[0] * m_vals[1] if iter == 0 else m_vals[1]
			for org_idx, org_model in enumerate(models):
				with org_model as model:
					# set exchanges
					model = set_env(model, env_fluxes, iter, Mi)

					# run optim
					org_fluxes = run_sampling(model, org_idx, iter, org_fluxes, m_vals, rep_idx=rep_idx, obj_percent=objective_percent)
				
		# update fluxes
		env_fluxes = update_sampling_env(env_fluxes, org_fluxes, flow, rel_abund, iter, m_vals, Mi, rep_idx)


	return env_fluxes, org_fluxes, M

In [11]:
def input_validation(models, media, iters, flow, rel_abund, m_vals=None, obj_percent=None):
    if models is None or not isinstance(models, list):
        raise ValueError("models must be a list of cobra.Model objects.")
        
	if media is None or not isinstance(media, dict):
        raise ValueError("media must be a dictionary with reaction IDs as keys and flux values as values.")



TabError: inconsistent use of tabs and spaces in indentation (3623117223.py, line 5)

# Test Models

In [12]:
# model_pre_processing
mod_paths = ['../AGORA2_Models/Escherichia_coli_str_K_12_substr_MG1655.mat',
			 "../AGORA2_Models/Bacteroides_thetaiotaomicron_3731.mat"]
S_matrix = [] #list of models
# Load Models and Save in S vector
for i in range(len(mod_paths)):
	model = cb.io.load_matlab_model(mod_paths[i])
	S_matrix.append(model) #append models to list

# Define input environment f_0
# this should be defined as a pandas dataframe with columns "Reaction" and "LB"
# glucose minimal medium
# Define Medium Components
glc_min_med = ['EX_glc_D(e)','EX_so4(e)','EX_nh4(e)','EX_no3(e)','EX_pi(e)','EX_cys_L(e)',
			   'EX_mn2(e)','EX_cl(e)','EX_ca2(e)','EX_mg2(e)','EX_cu2(e)','EX_cobalt2(e)','EX_fe2(e)','EX_fe3(e)','EX_zn2(e)','EX_k(e)']
# Define medium uptake flux bounds
glc_min_med_flux = [-10,-100,-100,-100,-100,-100,
					-100,-100,-100,-100,-100,-100,-100,-100,-100,-100]

glc_f0 = pd.DataFrame(data={'Reaction': glc_min_med,'LB': glc_min_med_flux})
glc_f0 = dict(zip(glc_min_med, glc_min_med_flux))
print(glc_f0)

No defined compartments in model model. Compartments will be deduced heuristically using regular expressions.
Using regular expression found the following compartments:c, e, p
No defined compartments in model model. Compartments will be deduced heuristically using regular expressions.
Using regular expression found the following compartments:c, e, p


{'EX_glc_D(e)': -10, 'EX_so4(e)': -100, 'EX_nh4(e)': -100, 'EX_no3(e)': -100, 'EX_pi(e)': -100, 'EX_cys_L(e)': -100, 'EX_mn2(e)': -100, 'EX_cl(e)': -100, 'EX_ca2(e)': -100, 'EX_mg2(e)': -100, 'EX_cu2(e)': -100, 'EX_cobalt2(e)': -100, 'EX_fe2(e)': -100, 'EX_fe3(e)': -100, 'EX_zn2(e)': -100, 'EX_k(e)': -100}


In [13]:
f, F = iipfba(S_matrix, glc_f0, np.array([0.5, 0.5]).reshape((-1,1)))


Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6
Iteration: 7
Iteration: 8
Iteration: 9


In [14]:
f, F, M = iisampling(S_matrix, glc_f0, np.array([0.5, 0.5]).reshape((-1,1)), 
				  iters=10, m_vals=[10,10])

# print((f.loc[0,:].to_numpy()).sum(axis=1))
# print(f)
# print(F)

import pickle
with open('ii_sampling_061025.pkl', 'wb') as file:
    pickle.dump([f, F, M], file)

Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6
Iteration: 7
Iteration: 8
Iteration: 9


In [23]:
F.loc[0 ,: ,:][["biomass525"]]


biomass525
Iteration Run            
0         0      0.706174
          1      0.706173
          2      0.706651
          3      0.706407
          4      0.706174
...                   ...
9         95     0.419227
          96     0.419252
          97     0.419253
          98     0.419291
          99     0.419279

[1000 rows x 1 columns]

In [362]:
print(F.loc[0, :]["biomass525"])

Iteration
0    0.784637
1    0.505684
2    0.472349
3    0.465576
4    0.464291
5    0.463970
6    0.463890
7    0.463870
8    0.463865
9    0.463863
Name: biomass525, dtype: float64
